_________________

# Setup

In [ ]:
import os
import ee
import geemap

ee.Authenticate()
ee.Initialize(project='ee-projectname')

Map = geemap.Map()

In [ ]:
newGeo = ee.FeatureCollection() #region of interest
bands_spatial_temporal_image = ee.Image() #raster with training bands/features
trainingLoaded_all_final_water = ee.FeatureCollection() #reference labeled dataset

Map.centerObject(newGeo,8)
Map.addLayer(newGeo)

____________________

## Get features for the labeled reference dataset

In [ ]:
def add_lat_lon(feature):
    """
    Add longitude (x) and latitude (y) coordinates to ee.FeatureCollection()
    """
    geometry = ee.Geometry(feature.geometry())
    coords = geometry.coordinates()
    return feature.set({
      'x': coords.get(0),  # Longitude is the first coordinate
      'y': coords.get(1)    # Latitude is the second coordinate
    })

exportTable = True

if exportTable:
    trainingLoaded_all_final_water = trainingLoaded_all_final_water.map(add_lat_lon)
    
    bands_spatial_temporal = bands_spatial_temporal_image.sampleRegions(
        collection=trainingLoaded_all_final_water,
        properties=["C_ID_1", "x", "y"],
        scale=10
    )
    
    #Export feature collection to drive
    task = ee.batch.Export.table.toDrive(
        collection=bands_spatial_temporal,
        description='bands_spatial_temporal_dataset',
        fileFormat='CSV'
    )
    task.start()

____

## Get newGeo geoJSON

In [ ]:
export_task = ee.batch.Export.table.toDrive(
    collection=newGeo,
    description='newGeo',
    fileFormat='GeoJSON'
)

export_task.start()

____

## Fishnet and download feature raster by tile

In [ ]:
#geemap fishnet
fishnet = geemap.fishnet(newGeo.geometry(), rows=12, cols=16, delta=0.1)
Map.addLayer(fishnet, {}, 'features fishnet')
Map

In [ ]:
#download
directory_to_save = "local_directory"
os.makedirs(directory_to_save, exist_ok=True)

geemap.download_ee_image_tiles(
    bands_spatial_temporal_image, fishnet, prefix="trainingfeatures_", out_dir=directory_to_save, crs="EPSG:4326", scale=10
)
print(f"Feature tiles saved in {feature_directory}")